In [ ]:
import pandas as pd

df = pd.read_csv('template.csv')
df

In [ ]:
# columns_to_drop = [
#     'date', 'game_num','completion_info','forfeit_info','protest_info', 'day_of_week', 'attendance', 
#     'additional_info','length_in_outs','time_of_game_min','v_line_score', 'h_line_score',
#     'v_manager_id',	'v_manager_name','h_manager_id','h_manager_name',
#     'hp_umpire_id', 'hp_umpire_name',
#     '1b_umpire_id', '1b_umpire_name',
#     '2b_umpire_id', '2b_umpire_name',
#     '3b_umpire_id', '3b_umpire_name',
#     'lf_umpire_id', 'lf_umpire_name',
#     'rf_umpire_id', 'rf_umpire_name',
#     'winning_pitcher_id', 'winning_pitcher_name',
#     'losing_pitcher_id', 'losing_pitcher_name',
#     'saving_pitcher_id', 'saving_pitcher_name',
#     'v_at_bats','v_hits','v_doubles','v_triples','v_homeruns','v_rbi','v_sacrifice_hits','v_sacrifice_flies',
#     'v_hit_by_pitch','v_walks','v_intentional_walks','v_strikeouts','v_stolen_bases','v_caught_stealing',
#     'v_grounded_into_dp','v_catcher_interference','v_left_on_base','v_pitchers_used','v_individual_earned_runs',
#     'v_team_earned_runs','v_wild_pitches','v_balks','v_putouts','v_assists','v_errors','v_passed_balls',
#     'v_double_plays','v_triple_plays','h_at_bats','h_hits','h_doubles','h_triples','h_homeruns','h_rbi',
#     'h_sacrifice_hits','h_sacrifice_flies','h_hit_by_pitch','h_walks','h_intentional_walks','h_strikeouts',
#     'h_stolen_bases','h_caught_stealing','h_grounded_into_dp','h_catcher_interference','h_left_on_base',
#     'h_pitchers_used','h_individual_earned_runs','h_team_earned_runs','h_wild_pitches','h_balks','h_putouts',
#     'h_assists','h_errors','h_passed_balls','h_double_plays','h_triple_plays', 'gwrbi_batter_id','gwrbi_batter_name','acquisition_info',
#     'v_league', 'h_league','v_game_num', 'h_game_num'
# ]

# df = df.drop(columns=columns_to_drop)
# # Remove all columns from df whose names contain 'pos'
# df = df.loc[:, ~df.columns.str.contains('pos')]
# cols_with_id = [col for col in df.columns if 'id' in col and col != 'park_id']
# df = df.drop(columns=cols_with_id)
# df.head()

In [ ]:
# Use only if values are not entered yet
# Prompt user for new values for all columns in the first row
for col in df.columns:
    current_value = df.at[0, col]
    new_value = input(f"Enter new value for '{col}' (current: {current_value}): ")
    if new_value != "":
        # Convert to int if the column is of int type
        if df[col].dtype == 'int64':
            new_value = int(new_value)
        df.at[0, col] = new_value

df.head(1)


,v_team,h_team,v_score,h_score,day_night,park_id,v_starting_pitcher_name,h_starting_pitcher_name,v_player_1_name,v_player_2_name,...,v_player_9_name,h_player_1_name,h_player_2_name,h_player_3_name,h_player_4_name,h_player_5_name,h_player_6_name,h_player_7_name,h_player_8_name,h_player_9_name
0,SDN,LA,7,8,1,LOS03,Nick Pivetta,Dustin May,Fernando Tatis,Luis Arraez,...,Elias Díaz,Shohei Ohtani,Mookie Betts,Freddie Freeman,Teoscar Hernández,Will Smith,Max Muncy,Andy Pages,Tommy Edman,Hyeseong Kim


In [147]:
df['day_night'] = df['day_night'].map({'N': 0, 'D': 1})
df['day_night'].value_counts()

day_night
0    1
Name: count, dtype: int64

In [148]:
teams = ['LAN','SDN','COL','ARI','WAS','CIN','SLN','PIT','MIA','SFN','ANA','BAL','DET','CHA','NYA','HOU','MIN','KCA','CLE','OAK','BOS','SEA','TOR','TBA','CHN','TEX','MIL','NYN','ATL','PHI']
team_to_num = {team: idx for idx, team in enumerate(teams)}
team_id_df = pd.DataFrame(list(team_to_num.items()), columns=['Team', 'ID'])
#display(team_id_df)
# Create one-hot encoded columns for each team using team_id_df
one_hot = pd.DataFrame(0, index=df.index, columns=team_id_df['Team'])

for _, row in team_id_df.iterrows():
    team = row['Team']
    team_id = row['ID']
    one_hot.loc[df['v_team'] == team, team] = -1
    one_hot.loc[df['h_team'] == team, team] = 1

one_hot.head()
df_encoded = pd.concat([df, one_hot], axis=1)

In [149]:
# Add columns for each park in park_to_num (if not already present) and set to 0
park_to_num = {'SEO01': 0, 'PHO01': 1, 'CIN09': 2, 'LOS03': 3, 'MIA02': 4, 'SAN02': 5, 'BAL12': 6, 'CHI12': 7, 'HOU03': 8, 'KAN06': 9, 'OAK01': 10, 'SEA03': 11, 'STP01': 12, 'ARL03': 13, 'NYC20': 14, 'PHI13': 15, 'CHI11': 16, 'WAS11': 17, 'MIL06': 18, 'STL10': 19, 'MIN04': 20, 'ATL03': 21, 'DEN02': 22, 'PIT08': 23, 'SFO03': 24, 'ANA01': 25, 'DET05': 26, 'NYC21': 27, 'CLE08': 28, 'TOR02': 29, 'BOS07': 30, 'MEX02': 31, 'LON01': 32, 'BIR01': 33, 'WIL02': 34}

for park in park_to_num.keys():
    if park not in df_encoded.columns:
        df_encoded[park] = 0

df_encoded.head()

,v_team,h_team,v_score,h_score,day_night,park_id,v_starting_pitcher_name,h_starting_pitcher_name,v_player_1_name,v_player_2_name,...,ANA01,DET05,NYC21,CLE08,TOR02,BOS07,MEX02,LON01,BIR01,WIL02
0,LAN,SDN,8,7,0,SAN02,Lou Trivino,Dylan Cease,Shohei Ohtani,Mookie Betts,...,0,0,0,0,0,0,0,0,0,0


In [150]:
for idx, row in df_encoded.iterrows():
    park = row['park_id']
    if park in df_encoded.columns:
        df_encoded.at[idx, park] = 1
df_encoded = df_encoded.drop(columns=['v_team', 'h_team', 'park_id'])
df_encoded

,v_score,h_score,day_night,v_starting_pitcher_name,h_starting_pitcher_name,v_player_1_name,v_player_2_name,v_player_3_name,v_player_4_name,v_player_5_name,...,ANA01,DET05,NYC21,CLE08,TOR02,BOS07,MEX02,LON01,BIR01,WIL02
0,8,7,0,Lou Trivino,Dylan Cease,Shohei Ohtani,Mookie Betts,Freddie Freeman,Teoscar Hernandez,Max Muncy,...,0,0,0,0,0,0,0,0,0,0


In [151]:
df = df_encoded

In [152]:

# if applies #####################################################
df['h_team_wins'] = (df['h_score'] > df['v_score']).astype(int)
df.head()
drop_columns = ['v_score', 'h_score']
df = df.drop(columns=drop_columns)
df = df[['h_team_wins'] + [col for col in df.columns if col != 'h_team_wins']]
df.head()

,h_team_wins,day_night,v_starting_pitcher_name,h_starting_pitcher_name,v_player_1_name,v_player_2_name,v_player_3_name,v_player_4_name,v_player_5_name,v_player_6_name,...,ANA01,DET05,NYC21,CLE08,TOR02,BOS07,MEX02,LON01,BIR01,WIL02
0,0,0,Lou Trivino,Dylan Cease,Shohei Ohtani,Mookie Betts,Freddie Freeman,Teoscar Hernandez,Max Muncy,Andy Pages,...,0,0,0,0,0,0,0,0,0,0


In [153]:


df1 = pd.read_csv('bat_cleaned_2025.csv')
df2 = pd.read_csv('pitch_cleaned_2025.csv')
df3 = df


In [154]:
import unicodedata

def normalize_name(name):
    # Remove 'Jr.', strip, and reverse if comma present
    name = ' '.join(name.replace('Jr.', '').strip().split(', ')[::-1]).strip()
    # Normalize unicode to closest ASCII
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    return name

df1['player_name'] = df1['player_name'].apply(normalize_name)
df2['player_name'] = df2['player_name'].apply(normalize_name)

df1


,player_id,player_name,xwoba,k_percent,bb_percent,barrels_per_pa_percent,hardhit_percent,launch_speed,bat_speed
0,660271,Shohei Ohtani,0.447,24.9,11.1,13.804714,62.433862,95.5,74.6
1,571970,Max Muncy,0.371,24.2,15.2,7.786885,52.413793,91.2,69.5
2,605141,Mookie Betts,0.355,9.0,10.5,4.511278,36.619718,88.8,65.9
3,681624,Andy Pages,0.345,18.4,5.1,6.274510,36.979167,88.7,71.0
4,624424,Michael Conforto,0.336,25.8,14.0,5.429864,45.238095,90.6,71.9
5,669257,Will Smith,0.393,17.4,15.0,7.511737,45.070423,90.8,67.6
6,518692,Freddie Freeman,0.392,20.3,8.2,7.758621,46.341463,91.5,68.0
7,606192,Teoscar Hernandez,0.352,22.4,3.7,8.411215,40.384615,88.4,71.2
8,669242,Tommy Edman,0.341,17.4,3.6,5.128205,41.721854,88.4,68.7
9,571771,Enrique Hernandez,0.317,26.2,9.0,6.896552,40.425532,89.9,69.1


In [155]:
import unicodedata

def normalize_name(name):
    if isinstance(name, str):
        return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    return name

player_columns = [
    'v_starting_pitcher_name', 'h_starting_pitcher_name',
    'v_player_1_name', 'v_player_2_name', 'v_player_3_name', 'v_player_4_name',
    'v_player_5_name', 'v_player_6_name', 'v_player_7_name', 'v_player_8_name', 'v_player_9_name',
    'h_player_1_name', 'h_player_2_name', 'h_player_3_name', 'h_player_4_name',
    'h_player_5_name', 'h_player_6_name', 'h_player_7_name', 'h_player_8_name', 'h_player_9_name'
]

for col in player_columns:
    if col in df3.columns:
        df3[col] = df3[col].apply(normalize_name)

In [156]:
# Define the mapping dictionary for name transformations
name_mapping = {
    'C. J. Abrams': 'CJ Abrams',
    'Michael Harris': 'Michael Harris II',
    'Josh Palacios': 'Joshua Palacios',
    'Gerardo Perdomo': 'Geraldo Perdomo',
    'Giovanny Urshela': 'Gio Urshela',
    'Yulieski Gurriel': 'Yuli Gurriel',
    'Michael Taylor': 'Michael A. Taylor',
    'Victor Scott': 'Victor Scott II',
    'Mike Siani': 'Michael Siani',
    'J. P. Martinez': 'J.D. Martinez',
    'Louie Varland': 'Louis Varland',
    'Mike Soroka': 'Michael Soroka',
    'J. P. France': 'J.P. France',
    'Daniel Lynch': 'Daniel Lynch IV',
    'Luis Ortiz': 'Luis F. Ortiz',
    'Matt Boyd': 'Matthew Boyd',
    'J. T. Ginn': 'J.T. Ginn',
    'Darren Scott McCaughan': 'Darren McCaughan',
    'Jake Junis': 'Jakob Junis'
    
}

# List of columns to update
columns_to_update = [
    'v_starting_pitcher_name', 'h_starting_pitcher_name',
    'v_player_1_name', 'v_player_2_name', 'v_player_3_name', 'v_player_4_name',
    'v_player_5_name', 'v_player_6_name', 'v_player_7_name', 'v_player_8_name', 'v_player_9_name',
    'h_player_1_name', 'h_player_2_name', 'h_player_3_name', 'h_player_4_name',
    'h_player_5_name', 'h_player_6_name', 'h_player_7_name', 'h_player_8_name', 'h_player_9_name'
]

# Apply the mapping to each specified column
for col in columns_to_update:
    if col in df3.columns:  # Check if column exists in df3
        df3[col] = df3[col].replace(name_mapping)

In [157]:
# Create mapping dictionaries for batters and pitchers
batter_name_to_id = dict(zip(df1['player_name'], df1['player_id']))
pitcher_name_to_id = dict(zip(df2['player_name'], df2['player_id']))

# List of columns for batters and pitchers
batter_columns = [
    'v_player_1_name', 'v_player_2_name', 'v_player_3_name', 'v_player_4_name', 'v_player_5_name',
    'v_player_6_name', 'v_player_7_name', 'v_player_8_name', 'v_player_9_name',
    'h_player_1_name', 'h_player_2_name', 'h_player_3_name', 'h_player_4_name', 'h_player_5_name',
    'h_player_6_name', 'h_player_7_name', 'h_player_8_name', 'h_player_9_name'
]
pitcher_columns = ['v_starting_pitcher_name', 'h_starting_pitcher_name']

# Replace batter names with IDs, print names that cannot be mapped
for col in batter_columns:
    mapped = df3[col].map(batter_name_to_id)
    missing = df3.loc[mapped.isna(), col].unique()
    for name in missing:
        if name not in batter_name_to_id:
            print(f"Batter name not mapped in column '{col}': {name}")
    # Ensure IDs are int (if possible), else keep original
    df3[col] = mapped.fillna(df3[col]).apply(lambda x: int(x) if isinstance(x, (int, float)) and not pd.isna(x) else x)

# Replace pitcher names with IDs, print names that cannot be mapped
for col in pitcher_columns:
    mapped = df3[col].map(pitcher_name_to_id)
    missing = df3.loc[mapped.isna(), col].unique()
    for name in missing:
        if name not in pitcher_name_to_id:
            print(f"Pitcher name not mapped in column '{col}': {name}")
    df3[col] = mapped.fillna(df3[col]).apply(lambda x: int(x) if isinstance(x, (int, float)) and not pd.isna(x) else x)

In [158]:
df3

,h_team_wins,day_night,v_starting_pitcher_name,h_starting_pitcher_name,v_player_1_name,v_player_2_name,v_player_3_name,v_player_4_name,v_player_5_name,v_player_6_name,...,ANA01,DET05,NYC21,CLE08,TOR02,BOS07,MEX02,LON01,BIR01,WIL02
0,0,0,642152,656302,660271,605141,518692,606192,571970,681624,...,0,0,0,0,0,0,0,0,0,0


In [159]:
df3.to_csv('test.csv', index=False)

In [160]:


df = df3
bat_cleaned = pd.read_csv('bat_cleaned_2025.csv')
pitch_cleaned = pd.read_csv('pitch_cleaned_2025.csv')
df

,h_team_wins,day_night,v_starting_pitcher_name,h_starting_pitcher_name,v_player_1_name,v_player_2_name,v_player_3_name,v_player_4_name,v_player_5_name,v_player_6_name,...,ANA01,DET05,NYC21,CLE08,TOR02,BOS07,MEX02,LON01,BIR01,WIL02
0,0,0,642152,656302,660271,605141,518692,606192,571970,681624,...,0,0,0,0,0,0,0,0,0,0


In [161]:
# Get xwOBA for visiting team
v_player_cols = [f'v_player_{i}_name' for i in range(1, 10)]
df['v_batting_xwoba'] = df[v_player_cols].apply(
    lambda row: bat_cleaned.set_index('player_id').loc[row, 'xwoba'].mean(), axis=1
)

# Get xwOBA for home team
h_player_cols = [f'h_player_{i}_name' for i in range(1, 10)]
df['h_batting_xwoba'] = df[h_player_cols].apply(
    lambda row: bat_cleaned.set_index('player_id').loc[row, 'xwoba'].mean(), axis=1
)

# Calculate team averages for additional batting stats
bat_stats = ['k_percent', 'bb_percent', 'barrels_per_pa_percent', 'hardhit_percent', 'launch_speed', 'bat_speed']

for stat in bat_stats:
    # Visiting team
    df[f'v_batting_{stat}'] = df[v_player_cols].apply(
        lambda row: bat_cleaned.set_index('player_id').loc[row, stat].mean(), axis=1
    )
    # Home team
    df[f'h_batting_{stat}'] = df[h_player_cols].apply(
        lambda row: bat_cleaned.set_index('player_id').loc[row, stat].mean(), axis=1
    )

In [162]:
# Get xwOBA for visiting starting pitcher
df['v_pitching_xwoba'] = df['v_starting_pitcher_name'].apply(
    lambda pid: pitch_cleaned.set_index('player_id').loc[pid, 'xwoba']
)
# Get xwOBA for home starting pitcher
df['h_pitching_xwoba'] = df['h_starting_pitcher_name'].apply(
    lambda pid: pitch_cleaned.set_index('player_id').loc[pid, 'xwoba']
)

# Pitching stats to aggregate
pitch_stats = [
    'k_percent', 'bb_percent', 'hardhit_percent',
    'barrels_per_bbe_percent', 'velocity', 'spin_rate', 'swing_miss_percent'
]

for stat in pitch_stats:
    df[f'v_pitching_{stat}'] = df['v_starting_pitcher_name'].apply(
        lambda pid: pitch_cleaned.set_index('player_id').loc[pid, stat]
    )
    df[f'h_pitching_{stat}'] = df['h_starting_pitcher_name'].apply(
        lambda pid: pitch_cleaned.set_index('player_id').loc[pid, stat]
    )

In [163]:
cols_to_round = [
    'v_batting_xwoba', 'h_batting_xwoba',
    'v_batting_k_percent', 'h_batting_k_percent',
    'v_batting_bb_percent', 'h_batting_bb_percent',
    'v_batting_barrels_per_pa_percent', 'h_batting_barrels_per_pa_percent',
    'v_batting_hardhit_percent', 'h_batting_hardhit_percent',
    'v_batting_launch_speed', 'h_batting_launch_speed',
    'v_batting_bat_speed', 'h_batting_bat_speed',
    'v_pitching_xwoba', 'h_pitching_xwoba',
    'v_pitching_k_percent', 'h_pitching_k_percent',
    'v_pitching_bb_percent', 'h_pitching_bb_percent',
    'v_pitching_hardhit_percent', 'h_pitching_hardhit_percent',
    'v_pitching_barrels_per_bbe_percent', 'h_pitching_barrels_per_bbe_percent',
    'v_pitching_velocity', 'h_pitching_velocity',
    'v_pitching_spin_rate', 'h_pitching_spin_rate',
    'v_pitching_swing_miss_percent', 'h_pitching_swing_miss_percent'
]
df[cols_to_round] = df[cols_to_round].round(4)

In [164]:
df

,h_team_wins,day_night,v_starting_pitcher_name,h_starting_pitcher_name,v_player_1_name,v_player_2_name,v_player_3_name,v_player_4_name,v_player_5_name,v_player_6_name,...,v_pitching_hardhit_percent,h_pitching_hardhit_percent,v_pitching_barrels_per_bbe_percent,h_pitching_barrels_per_bbe_percent,v_pitching_velocity,h_pitching_velocity,v_pitching_spin_rate,h_pitching_spin_rate,v_pitching_swing_miss_percent,h_pitching_swing_miss_percent
0,0,0,642152,656302,660271,605141,518692,606192,571970,681624,...,23.3333,39.779,10.0,11.0497,88.8,91.1,2203,2618,25.8,33.7


In [165]:
df = df.drop(columns=[col for col in df.columns if 'name' in col])
df

,h_team_wins,day_night,LAN,SDN,COL,ARI,WAS,CIN,SLN,PIT,...,v_pitching_hardhit_percent,h_pitching_hardhit_percent,v_pitching_barrels_per_bbe_percent,h_pitching_barrels_per_bbe_percent,v_pitching_velocity,h_pitching_velocity,v_pitching_spin_rate,h_pitching_spin_rate,v_pitching_swing_miss_percent,h_pitching_swing_miss_percent
0,0,0,-1,1,0,0,0,0,0,0,...,23.3333,39.779,10.0,11.0497,88.8,91.1,2203,2618,25.8,33.7


In [166]:
df.to_csv('2025_06_10_LA_SDN_cleaned.csv', index=False)